# Iterative Solvers Stability (Advanced)

## Objective

Study stability of iterative methods:
- Jacobi and Gauss-Seidel
- Conjugate Gradient
- Convergence and conditioning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
plt.rcParams["figure.figsize"] = (12, 6)

## Jacobi Iteration

For Ax = b, iterate: $x^{(k+1)} = D^{-1}(b - (L+U)x^{(k)})$

In [ ]:
def jacobi(A, b, max_iter=100, tol=1e-10):
    """Jacobi iteration."""
    n = len(b)
    x = np.zeros(n)
    D = np.diag(np.diag(A))
    R = A - D
    
    errors = []
    for k in range(max_iter):
        x_new = np.linalg.solve(D, b - R @ x)
        errors.append(np.linalg.norm(A @ x_new - b))
        if errors[-1] < tol:
            break
        x = x_new
    
    return x, errors

# Test on diagonally dominant matrix
n = 50
A = np.random.randn(n, n)
A = A + 10 * np.eye(n)  # Make diagonally dominant
b = np.random.randn(n)

x_jacobi, errors_jacobi = jacobi(A, b)

plt.figure(figsize=(10, 6))
plt.semilogy(errors_jacobi, linewidth=2)
plt.xlabel("Iteration")
plt.ylabel("Residual norm")
plt.title("Jacobi Convergence")
plt.grid(True, alpha=0.3)
plt.savefig("../plots/09_jacobi_convergence.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"Converged in {len(errors_jacobi)} iterations")
print(f"Final residual: {errors_jacobi[-1]:.2e}")

## Conjugate Gradient

For symmetric positive definite systems, CG is optimal Krylov method.

In [ ]:
def conjugate_gradient(A, b, max_iter=None, tol=1e-10):
    """Conjugate gradient method."""
    n = len(b)
    if max_iter is None:
        max_iter = n
    
    x = np.zeros(n)
    r = b - A @ x
    p = r.copy()
    
    errors = []
    
    for k in range(max_iter):
        Ap = A @ p
        alpha = np.dot(r, r) / np.dot(p, Ap)
        x = x + alpha * p
        r_new = r - alpha * Ap
        
        errors.append(np.linalg.norm(r_new))
        
        if errors[-1] < tol:
            break
        
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        p = r_new + beta * p
        r = r_new
    
    return x, errors

# Test on SPD matrix
n = 50
A = np.random.randn(n, n)
A = A.T @ A + np.eye(n)  # Make SPD
b = np.random.randn(n)

x_cg, errors_cg = conjugate_gradient(A, b)

plt.figure(figsize=(10, 6))
plt.semilogy(errors_cg, linewidth=2)
plt.xlabel("Iteration")
plt.ylabel("Residual norm")
plt.title("Conjugate Gradient Convergence")
plt.grid(True, alpha=0.3)
plt.savefig("../plots/09_cg_convergence.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"CG converged in {len(errors_cg)} iterations")
print(f"Theoretical max: {n} iterations")

## Key Takeaways

1. **Iterative methods** useful for large sparse systems
2. **Convergence** depends on matrix properties (diagonal dominance, SPD)
3. **CG is optimal** for SPD systems
4. **Preconditioning** can dramatically improve convergence